In [ ]:
import numpy as np

import skimage
import skimage.io
import skimage.transform

from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from quilt.data.jared import landuse_austin_tx

In [ ]:
def findimg(attop,root,layer) :
    atdir = getattr(attop,root)
    atimg = getattr(atdir,root+layer)
    return atimg

In [ ]:
def prepimage(attop,root): # prepare the image stack
    
    tmpimg = findimg(attop,root,'_B4')
    r=skimage.io.imread(tmpimg(), as_grey=True) # load red
    tmpimg = findimg(attop,root,'_B3')
    g=skimage.io.imread(tmpimg(), as_grey=True) # load green
    tmpimg = findimg(attop,root,'_B2')
    b=skimage.io.imread(tmpimg(), as_grey=True) # load blue
    tmpimg = findimg(attop,root,'_B5')
    ir=skimage.io.imread(tmpimg(), as_grey=True)# load band 5 - near infrared
    
    ndvi = np.true_divide((ir - r), (ir + r)) # ndvi 
    rgb= np.dstack((r,g,b))                   # combine into ordered stack

    tmpimg = findimg(attop,root,'_B8') # load hi-res image
    pan=skimage.io.imread(tmpimg(), as_grey=True)/65535 # load chromatic, normalize to between 0 and 1

    rgb_big=skimage.transform.resize(rgb, output_shape=(pan.shape[0],pan.shape[1],3), order=3, mode='constant', cval=0.0) # resize the rgb composite to match the chromatic
    ndvi_big=skimage.transform.resize(ndvi, output_shape=(pan.shape[0],pan.shape[1]), order=3, mode='constant', cval=0.0) # resize ndvi to match the rgb

    hsv = skimage.color.rgb2hsv(rgb_big) #Convert the rgb to the HSV colorspace
    hsv[...,2]=pan                       #Pan Sharpen
    rgb_pan = skimage.color.hsv2rgb(hsv) #Convert back to rgb space
    
    return rgb_pan, ndvi_big 

In [ ]:
attop = landuse_austin_tx.images.cropped              # use precropped images 
rgb, ndvi = prepimage(attop,'LC80270392014022LGN00')  # test image

In [ ]:
from scipy import ndimage as ndi
rgb_dog = rgb - ndi.gaussian_filter(rgb, 15) # difference of gaussians

In [ ]:
from math import floor

stack=np.dstack((rgb, rgb_dog, ndvi))

depth=stack.shape[2]
edge=5
buff=floor(edge/2)
stack_window=skimage.util.view_as_windows(np.pad(stack,pad_width=((buff,buff), (buff,buff),(0,0)), mode ='minimum'), window_shape=(edge,edge,depth), step=1) 
print('stack_window.shape ')
print(stack_window.shape) #5 by 5 moving window around each pixel

stack_flat=stack_window.reshape(-1,edge*edge*depth) 
print('stack_flat.shape')
print(stack_flat.shape) # flatten to rows

In [ ]:
#make labels
attop = landuse_austin_tx.labels.austin_rasters_cropped
qtlabel = getattr(attop,'austin_tx_buildings_labels_allt') # get 'all touched' lables
building= skimage.io.imread(qtlabel(), as_grey=True)       # load buildings

attop = landuse_austin_tx.labels.austin_rasters_cropped
qtlabel = getattr(attop,'austin_tx_landuse_labels_noallt') # get 'no all touched' lables
landuse = skimage.io.imread(qtlabel(), as_grey=True)       # load landuse

building=skimage.img_as_uint(building)
landuse=skimage.img_as_uint(landuse)

In [ ]:
ll = landuse.flatten() 
lb = building.flatten() 

In [ ]:
nodata = 55537 #the zero value

uselabinter = True # interpolate the trees 

ll[ll != nodata] = 1.
lb[lb != nodata] = 2.
ll[ll == nodata] = 0.

if uselabinter == False :
    labels = ll+lb
    labels[labels==3] = 2. # prefer building when overlap

if uselabinter == True :
    lb[lb == nodata] = 1.
    labels = lb

print(labels[labels!=0].shape)

skimage.io.imshow(labels.reshape(ndvi.shape))
plt.show()

In [ ]:
x_tmp=stack_flat
y_tmp=labels

x_tmp = stack_flat[labels>0,...]
y_tmp = labels[labels>0]

y_tmp = y_tmp.astype(int)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_tmp, y_tmp, test_size=2000, train_size=1000, random_state=42)


In [ ]:
tpot = TPOTClassifier(generations=80, population_size=50, verbosity=2, n_jobs=15)
tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))
tpot.export('tpot_landuse_pipeline.py')